## Setup

In [ ]:
# Imports 
import torch
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import functools
import torch_geometric as tg
import networkx as nx

from src.dataset_utils import theta_ds_create
from src.dataset_utils import S_ds_read_given_rows_batch, S_ds_compute

from src.phi import JTFS_forward
from src.jacobian import M_factory
from src.distances import distance_factory
from src.ftm import rectangular_drum
from src.ftm import constants as FTM_constants

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# KNN-G Parameters

n_hubs = 100
k = 5

## Creating KNN-G

### Dataset

In [ ]:
# Choose to read or create the parameters graphset and set the path according to it

read_dataset = False

if read_dataset:
    DatasetPath = "data/precompute_S/param_dataset.csv"
    S_DatasetPath = "data/precompute_S/S_dataset_full.parquet"
else:
    DatasetPath = "data/default_parameters.csv"

In [ ]:
# Reading/Creating the dataset

if read_dataset:
    DF = torch.from_numpy(pd.read_csv(DatasetPath).to_numpy()).to(device).to(torch.float)
else:
    bounds = [['omega', 'tau', 'p', 'd', 'alpha'],[(2.4, 3.8),(0.4, 3),(-5, -0.7),(-5, -0.5),(10e-05, 1)]]
    logscale = True
    DF = torch.from_numpy(theta_ds_create(bounds=bounds, subdiv=5, path='data/default_parameters.csv').to_numpy()).to(device).to(torch.float)

### Hubs

In [ ]:
# Choosing the initial hubs

n_dataset = DF.size(dim=0)
Id_hub = torch.linspace(0, n_dataset-1, steps=n_hubs, device=device).long()

Id_hub

In [ ]:
# Read/Compute the S(hubs)  

#Can't read too many indices at once or else the RAM will be filled very fast, so we do it in batch (1 batch = 35s so less than 70 per batch = slower than computing)
enough_ram = False 

if read_dataset and enough_ram:
    batch_size = 70
    S_hub = S_ds_read_given_rows_batch(S_DatasetPath, Id_hub, batch_size)
else:
    phi = JTFS_forward
    def S(theta):
        return phi(rectangular_drum(theta, logscale, **FTM_constants))
    S_hub = S_ds_compute(DF,Id_hub,S)

S_hub

In [ ]:
# Compute the M(hubs)

phi = JTFS_forward
M = M_factory(logscale,phi)

M_hub = torch.zeros(Id_hub.size(dim=0),DF.size(dim=1),DF.size(dim=1)).to(device)

for i in tqdm.tqdm(range(Id_hub.size(dim=0)), desc="Computing M", leave=True):
    M_hub[i,:,:] = M(DF[Id_hub[i],:])

M_hub[0,:,:]

### Allocation

In [ ]:
distance_PNP = distance_factory('PNP')

def F(i,j,h):
    """
    i dans [0,DF.size(dim=0)-1]
    j dans [0,DF.size(dim=0)-1]
    h dans [0,Id_hub.size(dim=0)-1]
    """
    return distance_PNP(DF[i,:],DF[j,:],M_hub[h,:,:])

In [ ]:
# Allocation of each point

Allocation = torch.zeros(DF.size(dim=0)).to(int).to(device)

for i in tqdm.tqdm(range(DF.size(dim=0)),desc='Allocating',leave=True):

    dmin = torch.inf
    argmin = None

    for k in range(Id_hub.size(dim=0)):
        d = F(i,Id_hub[k],k)
        if d<dmin:
            dmin = d
            argmin = k
        
    Allocation[i] = argmin

Allocation

### Graph from KNN

In [ ]:
# Distance with hubs

distance_PNP = distance_factory('PNP')

def D(theta_c,M_c,theta_r,M_r):
    """
    Average of both ways to solve problem with distance across hubs
    """
    return (distance_PNP(theta_c,theta_r,M_r)+distance_PNP(theta_r,theta_c,M_c))/2

def all_D(i):
    """
    i in [0,DF.size(dim=0)]
    """
    M_i = M_hub[Allocation[i]]
    T_M_j = torch.zeros(DF.size(dim=0),DF.size(dim=1),DF.size(dim=1)).to(device)
    for j in range(DF.size(dim=0)):
        T_M_j[j] = M_hub[Allocation[j]]
 
    all_D_vmap = torch.func.vmap(functools.partial(D,theta_r=DF[i,:],M_r=M_i),in_dims=(0,0))

    return all_D_vmap(DF, T_M_j)

In [ ]:
# Edge index 

def Knn_edge():
    """
    edge_index : 1 if one point in neighbour of the other OR vice-versa
    edge_attr : w(i,j) = 1/D(i,j)
    """
    COO_edges = [[],[]]

    edge_attr = []

    for i in tqdm.tqdm(range(DF.size(dim=0)),desc='Finding neighbours',leave=True):

        T_dist = all_D(i)

        T_dist_sorted,i_c_sorted = torch.sort(T_dist)

        for j in i_c_sorted[0:k+1]:
            if i!=j: #No self connections 
                COO_edges[0].append(i)
                COO_edges[1].append(j)
                COO_edges[1].append(i)
                COO_edges[0].append(j)
        
                edge_attr.append(1/T_dist_sorted[j])
                edge_attr.append(1/T_dist_sorted[j])

    return torch.tensor(COO_edges, dtype=torch.long).to(device),edge_attr



In [ ]:
# Create Graph Data object

edge_index,edge_attr = Knn_edge()

graph = tg.data.Data(x=DF, edge_index=edge_index, edge_attr=edge_attr)

## Visualisation of the graph

In [ ]:
# Gather some statistics about the graph.
print(f'Number of nodes: {graph.num_nodes}')
print(f'Number of edges: {graph.num_edges}')
print(f'Average node degree: {graph.num_edges / graph.num_nodes:.2f}')
print(f'Has isolated nodes: {graph.has_isolated_nodes()}')
print(f'Has self-loops: {graph.has_self_loops()}')
print(f'Is undirected: {graph.is_undirected()}')

In [ ]:
def visualize_graph(G,pos):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G,pos=pos,with_labels=False,node_size=3)
    plt.show()

In [ ]:
G = tg.utils.to_networkx(graph, to_undirected=True)

nx.write_graphml(G, 'data/Knn-G/knnG.graphml')

pos = nx.forceatlas2_layout(G,gravity=0.1)
#pos = nx.spring_layout(G)
visualize_graph(G,pos)